# Exercises for Lecture 15 (Deep CNN architectures)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from functools import partial

# To make this notebook's output stable across runs
def reset_state(seed=42):
    tf.keras.backend.clear_session()
    tf.random.set_seed(seed)
    np.random.seed(seed)

2024-01-10 00:30:31.233273: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-10 00:30:31.284336: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-10 00:30:31.285454: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-01-10 00:30:32.130193: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Exercise 1: Build a ResNet CNN architecture for fashion MNIST.

Load and set up data.

In [2]:
# Load fashion MNIST data
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-30000], X_train_full[-30000:]
y_train, y_valid = y_train_full[:-30000], y_train_full[-30000:]

# Standardize
X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

# Add final channel axis (one channel)
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

Use the subclassing API to define a `ResidualUnit` layer.

In [3]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="SAME", use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

Buid a ResNet model using the layer you defined above.

In [4]:
model = keras.models.Sequential()
model.add(DefaultConv2D(64, kernel_size=7, strides=2,
                        input_shape=[28, 28, 1]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=2, strides=2, padding="SAME"))
prev_filters = 64
for filters in [64] * 2 + [128] * 2:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))

In [5]:
model.summary()

Model: "sequential"


_________________________________________________________________


 Layer (type)                Output Shape              Param #   


 conv2d (Conv2D)             (None, 14, 14, 64)        3136      


 batch_normalization (Batch  (None, 14, 14, 64)        256       


...

 dense (Dense)               (None, 10)                1290      


Total params: 680010 (2.59 MB)


Trainable params: 678090 (2.59 MB)


Non-trainable params: 1920 (7.50 KB)


_________________________________________________________________


Compile your model and train it.

In [6]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [7]:
history = model.fit(X_train, y_train, epochs=3, validation_data=(X_valid, y_valid))

Epoch 1/3


  1/938 [..............................] - ETA: 1:08:41 - loss: 3.1139 - accuracy: 0.0625

  2/938 [..............................] - ETA: 49s - loss: 2.6676 - accuracy: 0.1562    

  3/938 [..............................] - ETA: 50s - loss: 2.6567 - accuracy: 0.1562

  4/938 [..............................] - ETA: 49s - loss: 2.3003 - accuracy: 0.2812

...

934/938 [============================>.] - ETA: 0s - loss: 0.2704 - accuracy: 0.8991

935/938 [============================>.] - ETA: 0s - loss: 0.2702 - accuracy: 0.8992

936/938 [============================>.] - ETA: 0s - loss: 0.2703 - accuracy: 0.8992

937/938 [============================>.] - ETA: 0s - loss: 0.2703 - accuracy: 0.8991

938/938 [==============================] - 56s 60ms/step - loss: 0.2703 - accuracy: 0.8991 - val_loss: 0.3324 - val_accuracy: 0.8823


Evaluate the model performance on the test set.

In [8]:
model.evaluate(X_test, y_test)

  1/313 [..............................] - ETA: 7s - loss: 0.3665 - accuracy: 0.8750

  6/313 [..............................] - ETA: 3s - loss: 0.3055 - accuracy: 0.8854

 12/313 [>.............................] - ETA: 2s - loss: 0.3034 - accuracy: 0.8880

 18/313 [>.............................] - ETA: 2s - loss: 0.3074 - accuracy: 0.8889

 24/313 [=>............................] - ETA: 2s - loss: 0.3403 - accuracy: 0.8776

...

300/313 [===========================>..] - ETA: 0s - loss: 0.3531 - accuracy: 0.8722

306/313 [============================>.] - ETA: 0s - loss: 0.3515 - accuracy: 0.8723

312/313 [============================>.] - ETA: 0s - loss: 0.3504 - accuracy: 0.8725

313/313 [==============================] - 3s 9ms/step - loss: 0.3501 - accuracy: 0.8726


[0.3501233160495758, 0.8726000189781189]